In [ ]:
import numpy as np
from sympleq.core.paulis import PauliSum, PauliString
from sympleq.core.circuits.target import find_map_to_target_pauli_sum
from sympleq.core.circuits import Gate, Circuit, SWAP
from sympleq.utils import get_linear_dependencies
from sympleq.graph_utils import find_one_permutation, permutation_to_swaps, mapping_key, brute_force_all_permutations, find_swapped_dependent_elements
from sympleq.models import ToricCode, Hadamard_Symmetric_PauliSum, SWAP_symmetric_PauliSum
from scripts.experiments.symmetries.src.pauli import symplectic_pauli_reduction 
from scripts.experiments.symmetries.src.permutations import find_first_automorphism

In [ ]:
seed = None

d = 2
n_qubits = 5
n_sym_q = 2
n_paulis = 12
H, C = Hadamard_Symmetric_PauliSum(n_paulis, n_qubits, n_sym_q, seed=seed)
H.combine_equivalent_paulis()

In [ ]:
independent_paulis, dependencies = get_linear_dependencies(H.tableau, d)

print(independent_paulis)
print(dependencies)

In [ ]:
cs = H.weights

graph_dict = {}

for i in independent_paulis:
    key = cs[i]
    if key in graph_dict:
        graph_dict[key].append([i])
    else:
        graph_dict[key] = [[i]]

for i in dependencies.keys():
    key = cs[i]
    dependency = dependencies[i]
    dependence_indices = [x[0] for x in dependency]
    dependence_multiplicities = [x[1] for x in dependency]  # this will be needed for qudits! always 1 for now
    if key in graph_dict:
        graph_dict[key].append(dependence_indices)
    else:
        graph_dict[key] = [dependence_indices]

print(graph_dict[1])

In [ ]:
def checker(permutation):
    pairs = permutation_to_swaps(permutation)
    swapped_dependents = find_swapped_dependent_elements(pairs, graph_dict[1])
    H_target = H.copy()
    for p in swapped_dependents:
        H_target.swap_paulis(p[0], p[1])
    return np.array_equal(H_target.symplectic_product_matrix(), H.symplectic_product_matrix())


In [ ]:
n = len(independent_paulis)
vectors = graph_dict[1]
automorphism = find_first_automorphism(vectors, n, checker)
print(automorphism)


In [ ]:
H_t = H.copy()
H_t = H_t[automorphism]
H_i = H[independent_paulis]

F, h, _, _ = find_map_to_target_pauli_sum(H_i, H_t)


In [ ]:
G = Gate('Symmetry', [i for i in range(H.n_qudits())], F.T, 2, h)
print(F)
